# LLama-2 Transformer

> Geenral architecture for transformers.

In [ ]:
#| default_exp models.program.synthesizer

In [ ]:
#| export
from fastcore import *
from fastcore.utils import *

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
from mawm.models.base.dense import DenseModel


class Proposer(nn.Module):
    """
    Autoregressive Synthesizer that generates (primitive, params)
    conditioned on observation latent z and previous program token.
    """

    def __init__(
        self,
        obs_dim,
        num_prims,
        max_params,
        seq_len,
        prog_emb_dim_x=64,
        prog_emb_dim_y=64,
        prog_emb_dim_prims=32,
        model_info={ 'layers': 3,'node_size': 128,'activation': nn.ReLU,'dist': None}
    ):
        super().__init__()

        # observation -> hidden
        self.obs_proj = nn.Linear(obs_dim, model_info['node_size'])
        self.prog_embed = nn.Linear(seq_len * (prog_emb_dim_prims + prog_emb_dim_x + prog_emb_dim_y), model_info['node_size'])

        # combine obs + prev token
        self.fuse = nn.Linear(model_info['node_size'] + model_info['node_size'],
                              model_info['node_size'])
        self.proposer_mlp = DenseModel(output_shape= (model_info['node_size'],), input_size=model_info['node_size'], info= model_info)

        # heads
        self.prim_head = nn.Linear(model_info['node_size'], num_prims+ 1)  # +1 for EOS
        self.param_head = nn.Linear(model_info['node_size'], max_params)
        self.max_params = max_params
        self.num_prims = num_prims
        self.sos_idx = -1 #num_prims

    def forward_step(self, z, p_embed):
        """
        z:              (B, obs_dim)
        p_embed:        (B, L, prog_embed_dim) - where prog_embed_dim = prog_emb_dim_prims + prog_emb_dim_x + prog_emb_dim_y
        """
        # import ipdb; ipdb.set_trace()
        z_h = F.relu(self.obs_proj(z))      # z_h shape: (B, H)
        
        B, L, dim = p_embed.shape
        program = p_embed.view(B, L*dim)  # program shape: (B, L * prog_embed_dim)
        program = F.relu(self.prog_embed(program)) # program shape: (B, L, H)

        h = torch.cat([z_h, program], dim=-1)
        
        h = F.relu(self.fuse(h))
        h = self.proposer_mlp(h)

        prim_logits = self.prim_head(h)             
        param_logits = self.param_head(h)           
        param_probs = torch.sigmoid(param_logits)   

        return prim_logits, param_probs


In [ ]:
#| hide
from mawm.core import Program, PRIMITIVE_TEMPLATES

p = Program(tokens= [(0, [1.0, 2.0]), (4, []), (1, [3.0, 4.0])])

In [ ]:
#| hide
from mawm.models.program.embedder import batchify_programs, get_indices
from mawm.core import Program, PRIMITIVE_TEMPLATES
batch_programs = [
    Program(tokens= [(0, [1.0, 2.0]), (4, []), (1, [3.0, 4.0]),(5, [1]) ]), # L=4
    Program(tokens= [(2, [5.0]), (3, [6.0, 5.0])]) # L=2
]


prim_ids, params_ids = batchify_programs(batch_programs)



In [ ]:
prim_ids

tensor([[ 0,  4,  1,  5],
        [ 2,  3, -1, -1]])

In [ ]:
#| hide
from mawm.models.program.embedder import ProgramEmbedder
num_primitives = len(PRIMITIVE_TEMPLATES)
p_embed = ProgramEmbedder(
    num_primitives= num_primitives,
    param_cardinalities= [7, 7],
    max_params_per_primitive= 2,
    d_name= 32,
    d_param= 32,
)
p_vec = p_embed(prim_ids, params_ids)
p_vec.shape

torch.Size([2, 4, 96])

In [ ]:
synthesizer= Proposer(
    obs_dim= 32,
    num_prims= num_primitives,
    max_params= 2,
    seq_len= 4,
    prog_emb_dim_x= 32,
    prog_emb_dim_y= 32,
    prog_emb_dim_prims= 32,
)
a, b = synthesizer.forward_step(
    z= torch.randn(2, 32),
    p_embed= p_vec,
)

In [ ]:
a.shape, b.shape

(torch.Size([2, 6]), torch.Size([2, 2]))

In [ ]:
a[0]

tensor([-0.0553, -0.0427, -0.2560,  0.0010,  0.0799,  0.0142],
       grad_fn=<SelectBackward0>)

In [ ]:
b

tensor([[0.5361, 0.4981],
        [0.5012, 0.5060]], grad_fn=<SigmoidBackward0>)

In [ ]:
softmax_with_temperature = lambda logits, temp: F.softmax(logits / temp, dim=-1)
softmax_with_temp = softmax_with_temperature(a, temp=0.2)
softmax_with_temp

tensor([[0.0973, 0.1896, 0.0967, 0.2142, 0.1517, 0.2506]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
topk = 2
probs = F.softmax(a, dim=-1)
print(probs)
top_vals, top_idx = torch.topk(probs, k=min(topk, probs.size(0)), dim=-1)

tensor([[0.1512, 0.1728, 0.1510, 0.1771, 0.1653, 0.1827]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
top_vals, top_idx

(tensor([[0.1827]], grad_fn=<TopkBackward0>), tensor([[5]]))

In [ ]:
b

tensor([[0.4800, 0.5000]], grad_fn=<SigmoidBackward0>)

## Old trials

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional, Union

@dataclass
class ModelConfig:
    dim: int = 4096  # Dimension of the model
    n_layers: int = 32  # Number of layers in the transformer
    n_heads: int = 32  # Number of attention heads
    n_kv_heads: Optional[int] = n_heads  # Number of key-value heads (optional, defaults to n_heads)
    vocab_size: int = 50257  # Vocabulary size
    norm_eps: float = 1e-5  # Epsilon value for normalization

    max_batch_size: int = 32  # Maximum batch size for training
    max_seq_len: int = 2048  # Maximum sequence length

    device: str = None  # Device to run the model on (optional)


In [ ]:
#| export
class RotaryPositionEmbedding(nn.Module):

    def __init__(self, head_dim: int, seq_len: int, device: str) -> None:
        super().__init__()
        self.dim = head_dim
        assert self.dim % 2 == 0, "head_dim must be divisible by 2"

        # Calculate the rotation frequencies for positional embeddings
        theta_numerator = torch.arange(0, self.dim, 2, dtype=torch.float32)
        theta = 1.0 / torch.pow(10000, theta_numerator / self.dim).to(device)

        # Generate frequency values for positional embeddings
        m = torch.arange(seq_len, dtype=torch.float32).to(device)
        freqs = torch.outer(m, theta).float()

        # Convert frequency values to complex numbers (polar form)
        self.freqs_complex = torch.polar(torch.ones_like(freqs), freqs)
        # self.register_buffer("freqs_complex", self.freqs_complex)

    def forward(self, x: torch.Tensor, start_pos: int) -> torch.Tensor:
        batch_size, seq_len, dim = x.shape
        assert dim == self.dim, "dim must be equal to self.dim"

        # Reshape the input into a complex tensor for rotational operations
        # (B, SeqLen, H, Head_Dim) -> (B, SeqLen, H, Head_Dim // 2)
        x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))

        # Extract rotational frequencies for the given sequence length and start position
        # (SeqLen, Head_Dim // 2) -> (1, SeqLen, 1, Head_Dim // 2)
        freq_complex = self.freqs_complex[start_pos:start_pos + seq_len]
        freq_complex = freq_complex.unsqueeze(0).unsqueeze(2)

        # Apply rotational transformation to the input using frequency values
        # (B, SeqLen, H, Head_Dim // 2) * (1, SeqLen, 1, Head_Dim // 2) -> (B, SeqLen, H, Head_Dim // 2)
        x_rotated = x_complex * freq_complex

        # Convert the rotated complex tensor back to real-valued tensor
        # (B, SeqLen, H, Head_Dim // 2) -> (B, SeqLen, H , Head_Dim // 2, 2)
        x_out = torch.view_as_real(x_rotated)

        # Reshape to match the original input shape
        # (B, SeqLen, H , Head_Dim // 2, 2) -> (B, SeqLen, H, Head_Dim)
        x_out = x_out.reshape(*x.shape)

        return x_out.type_as(x).to(x.device)



In [ ]:
#| export
class RMSNorm(nn.Module):

    def __init__(self, dim: int, eps: float) -> None:
        super().__init__()
        self.eps = eps  # Epsilon value for numerical stability
        self.gamma = nn.Parameter(torch.ones(dim))  # Learnable parameter for scaling

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: Input tensor of shape (Batch_Size, SeqLen, Dim)

        # Calculate the root-mean-square norm along the last dimension
        rms = torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True) + self.eps)

        # Normalize the input by dividing by the root-mean-square norm and scale with gamma
        normalized_x = (x / rms) * self.gamma

        return normalized_x  # Return the normalized tensor


In [ ]:
#| export
class SelfAttention(nn.Module):

    def __init__(self, args: ModelConfig):
        super().__init__()

        # Determine the number of key-value heads (defaults to n_heads if not specified)
        self.n_kv_heads = args.n_kv_heads if args.n_kv_heads is not None else args.n_heads

        # Set the number of query heads and the number of repetitions for K and V
        self.n_heads_q = args.n_heads
        self.n_rep = self.n_heads_q // self.n_kv_heads

        # Calculate the head dimension
        self.head_dim = args.dim // args.n_heads

        # Linear transformations for queries, keys, values, and output
        self.Wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.Wk = nn.Linear(args.dim, args.n_kv_heads * self.head_dim, bias=False)
        self.Wv = nn.Linear(args.dim, args.n_kv_heads * self.head_dim, bias=False)
        self.Wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

        # Initialize key and value caches with zeros
        self.cache_k = torch.zeros((args.max_batch_size, args.max_seq_len, args.n_kv_heads, self.head_dim))
        self.cache_v = torch.zeros((args.max_batch_size, args.max_seq_len, args.n_kv_heads, self.head_dim))

        # Rotary Position Embedding
        self.rope = RotaryPositionEmbedding(self.head_dim, args.max_seq_len, args.device)

    @staticmethod
    def repeat_heads(x: torch.Tensor, n_rep: int) -> torch.Tensor:

        # Repeat the heads of K and V to match the number of heads in Q

        batch_size, seq_len, n_kv_heads, head_dim = x.shape
        if n_rep == 1:
            return x
        else:
            return (x[:, :, :, None, :]
                    .expand(batch_size, seq_len, n_kv_heads, n_rep, head_dim)
                    .reshape(batch_size, seq_len, n_kv_heads * n_rep, head_dim)
                    )

    def forward(self, x: torch.Tensor, start_pos: int) -> torch.Tensor:
        batch_size, seq_len, dim = x.shape  # (B, 1, dim)
        assert dim == self.dim, "dim must be equal to self.dim"

        # (B, 1, dim) -> (B, 1, n_heads_q * head_dim)
        xq = self.Wq(x)

        # (B, 1, dim) -> (B, 1, n_kv_heads * head_dim)
        xk = self.Wk(x)

        # (B, 1, dim) -> (B, 1, n_kv_heads * head_dim)
        xv = self.Wv(x)

        # (B, 1, n_heads_q * head_dim) -> (B, 1, n_heads_q, head_dim)
        xq = xq.view(batch_size, seq_len, self.n_heads_q, self.head_dim)

        # (B, 1, n_kv_heads * head_dim) -> (B, 1, n_kv_heads, head_dim)
        xk = xk.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)
        xv = xv.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)

        xq = self.rope(xq, start_pos)
        xk = self.rope(xk, start_pos)

        # Update key and value caches
        self.cache_k[:batch_size, start_pos:start_pos + seq_len] = xk
        self.cache_v[:batch_size, start_pos:start_pos + seq_len] = xv

        # Retrieve key and value caches
        keys = self.cache_k[:batch_size, :start_pos + seq_len]
        values = self.cache_v[:batch_size, :start_pos + seq_len]

        # Repeat the heads of K and V to match the number of heads in Q
        keys = self.repeat_heads(keys, self.n_rep)
        values = self.repeat_heads(values, self.n_rep)

        # (B, 1, n_heads_q, head_dim) -> (B, n_heads_q, 1, head_dim)
        xq = xq.transpose(1, 2)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)

        # (B, n_heads_q, 1, head_dim) * (B, n_heads_q, head_dim, SeqLen) -> (B, n_heads_q, 1, SeqLen)
        scores = torch.matmul(xq, keys.transpose(-2, -1)) / math.sqrt(self.head_dim)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)

        # (B, n_heads_q, 1, SeqLen) * (B, n_heads_q, SeqLen, head_dim) -> (B, n_heads_q, 1, head_dim)
        context = torch.matmul(scores, values)

        # (B, n_heads_q, 1, head_dim) -> (B, 1, head_dim)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)

        # (B, 1, head_dim) -> (B, 1, dim)
        output = self.Wo(context)

        return output


In [ ]:
#| export
class FeedForward(nn.Module):

    def __init__(self, args: ModelConfig):
        super().__init__()

        # Calculate the hidden dimension based on the provided parameters
        hidden_dim = 4 * args.dim
        hidden_dim = int(2 * hidden_dim / 3)

        # Adjust the hidden dimension based on ffn_dim_multiplier (if provided)
        if args.ffn_dim_multiplier is not None:
            hidden_dim = int(args.ffn_dim_multiplier * hidden_dim)

        # Ensure hidden_dim is a multiple of args.multiple_of
        hidden_dim = args.multiple_of * ((hidden_dim + args.multiple_of - 1) // args.multiple_of)

        # Define linear layers for the feedforward network
        self.fc1 = nn.Linear(args.dim, hidden_dim, bias=False)
        self.fc2 = nn.Linear(hidden_dim, args.dim, bias=False)
        self.fc3 = nn.Linear(args.dim, hidden_dim, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Input shape: (Batch_Size, SeqLen, Dim)

        # Apply the first linear transformation and activation (swish)
        swish = F.silu(self.fc1(x))

        # Apply the second linear transformation
        x_V = self.fc3(swish)

        # Element-wise multiplication
        x = swish * x_V

        # Apply the third linear transformation
        x = self.fc2(x)

        return x  # Return the output


In [ ]:
#| export

class EncoderBlock(nn.Module):

    def __init__(self, args: ModelConfig):
        super().__init__()

        self.n_heads = args.n_heads
        self.dim = args.dim
        self.head_dim = args.dim // args.n_heads

        self.attention = SelfAttention(args)
        self.feed_forward = FeedForward(args)

        self.norm1 = RMSNorm(args.dim, args.norm_eps)
        self.ffn_norm = RMSNorm(args.dim, args.norm_eps)

    def forward(self, x: torch.Tensor, start_pos: int) -> torch.Tensor:
        h = x + self.attention(self.norm1(x), start_pos)
        out = h + self.feed_forward(self.ffn_norm(h))
        return out



In [ ]:
#| export
class Transformer(nn.Module):

    def __init__(self, args: ModelConfig) -> None:
        super().__init__()

        # Check if vocab_size is specified
        assert args.vocab_size != -1, "vocab_size must be specified"

        # Store model configuration and necessary parameters
        self.args = args
        self.vocab_size = args.vocab_size
        self.n_layers = args.n_layers

        # Embedding layer for token embeddings
        self.embeddings = nn.Embedding(self.vocab_size, args.dim)

        # Create a list of transformer encoder blocks
        self.layers = nn.ModuleList()
        for _ in range(args.n_layers):
            self.layers.append(EncoderBlock(args))

        # Layer normalization for the output
        self.norm = RMSNorm(args.dim, args.norm_eps)

        # Output linear layer
        self.output = nn.Linear(args.dim, self.vocab_size, bias=False)

    def forward(self, x: torch.Tensor, start_pos: int) -> torch.Tensor:
        # Input shape: (Batch_Size, SeqLen)

        # Ensure seq_len is 1
        assert x.shape[1] == 1, "seq_len must be 1"

        # Embedding lookup
        x = self.embeddings(x)

        # Pass through each transformer encoder block
        for layer in self.layers:
            x = layer(x, start_pos)

        # Layer normalization
        x = self.norm(x)

        # Output prediction
        x = self.output(x)

        return x  # Return the output
    

In [ ]:
# #| hide
# model_config = ModelConfig()
# model = Transformer(model_config)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()